In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold;

import xgboost as xgb

from sklearn.metrics import accuracy_score, f1_score

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', palette='husl')

from sklearn.preprocessing import normalize, scale

import warnings
warnings.filterwarnings('ignore')

from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from hyperopt.pyll.stochastic import sample

from sklearn.metrics import recall_score, precision_score

from sklearn.model_selection import cross_val_score #KFold

import keras
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils.np_utils import to_categorical


from IPython.display import Audio, display



path = "/home/steve/Desktop/challenge1_HowCrispCanYouClassify/data/"
train_data = pd.read_csv(path+"digits_train.csv")
test_data = pd.read_csv(path+"digits_test.csv")
train_data.head()

/home/steve/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,55,56,57,58,59,60,61,62,63,labels
0,0,0,0,2,12,3,0,0,0,0,...,0,0,0,1,15,16,13,4,0,6
1,1,0,0,13,13,13,12,4,0,0,...,0,0,1,9,14,15,4,0,0,5
2,2,0,0,0,12,8,0,6,3,0,...,0,0,0,0,12,6,0,0,0,4
3,3,0,0,1,11,10,0,0,0,0,...,0,0,0,0,13,8,0,0,0,9
4,4,0,0,1,10,14,13,2,0,0,...,0,0,0,0,14,2,0,0,0,7


In [2]:
print("The size of train_data: {0} and test_data {1}".format(
train_data.shape, test_data.shape))

The size of train_data: (4000, 66) and test_data (1620, 65)


In [3]:
train_data.drop(['Unnamed: 0'], axis=1, inplace=True)
train_data.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       '61', '62', '63', 'labels'],
      dtype='object')

In [4]:
test_data.drop(['Unnamed: 0'], axis=1, inplace=True)
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,12,16,9,0,0,0,0,3,...,16,0,0,1,10,15,16,16,10,0
1,0,2,11,16,5,0,0,0,0,10,...,10,0,0,2,12,13,12,10,1,0
2,0,0,9,15,14,1,0,0,0,2,...,12,0,0,0,9,16,16,16,6,0
3,0,0,6,15,16,12,1,0,0,3,...,6,0,0,0,6,14,12,4,0,0
4,0,0,11,13,12,12,3,0,0,5,...,0,0,0,1,12,16,12,2,0,0


In [5]:
"""
tr = [x for x in range(len(train_data)) if x % 10 != 0 ]
te = [x for x in range(len(train_data)) if x % 10 == 0]

train = train_data.loc[tr]
test = train_data.loc[te]
"""
train = train_data.copy()
test = test_data.copy()
test.shape

(1620, 64)

In [6]:
ntrain = train.shape[0]
ntest = test.shape[0]

NUMCLASS=len(np.unique(train['labels']))

SEED = 2
NFOLDS = 5
kf = KFold(ntrain, n_folds=NFOLDS, random_state=SEED, shuffle=True)
kf

sklearn.cross_validation.KFold(n=4000, n_folds=5, shuffle=True, random_state=2)

In [7]:
#COVNETS PARAMETERS
cnnmodel = Sequential() 
cnnmodel.add(Conv2D(48,3, 3, activation='relu', input_shape=(8,8, 1))) 
cnnmodel.add(Conv2D(48,3, 3, activation='relu')) 
cnnmodel.add(MaxPooling2D(pool_size=(1,1)))
cnnmodel.add(Conv2D(48,3, 3, activation='sigmoid'))  
cnnmodel.add(MaxPooling2D(pool_size=(1,1)))

cnnmodel.add(Dropout(0.33))


cnnmodel.add(Flatten())

cnnmodel.add(Dense(36, activation='sigmoid'))
cnnmodel.add(Dense(10, activation='softmax')) 
cnnmodel.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [8]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        
        if 'random_state' in params:
            params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
    
class XgbHelper(object):
    def __init__(self, seed=SEED, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)
        
    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gdbt = xgb.train(self.param, dtrain, self.nrounds)
        
    def predict(self, x):
        return self.gdbt.predict(xgb.DMatrix(x))

In [9]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
    
    list_vals = oof_test_skf.mean(axis=0)
    oof_test[:] = [round(x, 0) for x in list_vals]
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)



In [10]:
y_train = train['labels'].ravel()
train = train.drop(['labels'], axis=1)
x_train = train.values
train.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,2,12,3,0,0,0,0,0,...,12,0,0,0,1,15,16,13,4,0
1,0,0,13,13,13,12,4,0,0,1,...,1,0,0,1,9,14,15,4,0,0
2,0,0,0,12,8,0,6,3,0,0,...,0,0,0,0,0,12,6,0,0,0
3,0,0,1,11,10,0,0,0,0,0,...,0,0,0,0,0,13,8,0,0,0
4,0,0,1,10,14,13,2,0,0,0,...,0,0,0,0,0,14,2,0,0,0


In [11]:
"""
y_test = test['labels'].ravel()
test = test.drop(['labels'], axis=1)
x_test = test.values
print("The shape for train:{0} and test{1}".format(x_train.shape, x_test.shape))
"""
x_test = test.values
x_test[:3]

array([[ 0,  0, 12, 16,  9,  0,  0,  0,  0,  3, 16,  7,  5,  0,  0,  0,  0,
        11, 13,  0,  0,  0,  0,  0,  0,  9, 12,  0,  0,  0,  0,  0,  0,  9,
        13, 10, 15,  8,  0,  0,  0,  5, 16, 15,  8, 11, 10,  0,  0,  4, 16,
        12,  1,  5, 16,  0,  0,  1, 10, 15, 16, 16, 10,  0],
       [ 0,  2, 11, 16,  5,  0,  0,  0,  0, 10, 14, 12, 12,  0,  0,  0,  0,
         3,  2,  8, 11,  0,  0,  0,  0,  0,  0, 11, 12,  0,  0,  0,  0,  0,
         1, 13, 16, 10,  0,  0,  0,  0,  0,  0,  4, 16,  5,  0,  0,  0,  8,
         6,  6, 16, 10,  0,  0,  2, 12, 13, 12, 10,  1,  0],
       [ 0,  0,  9, 15, 14,  1,  0,  0,  0,  2, 16,  8, 15, 10,  0,  0,  0,
         2, 14,  4, 13, 14,  1,  0,  0,  0,  7, 15, 15, 14,  6,  0,  0,  0,
         0,  1,  1,  7,  9,  0,  0,  0,  0,  0,  0,  3, 12,  0,  0,  0,  6,
         0,  0,  6, 12,  0,  0,  0,  9, 16, 16, 16,  6,  0]])

In [12]:
ntrain = x_train.shape[0]
ntest = x_test.shape[0]
print("the ntrain size is {0} and ntest size is {1}".format(ntrain, ntest))

the ntrain size is 4000 and ntest size is 1620


In [13]:
KNeighborsClassifier()

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [14]:
#PARAMETER TUNING
def hyperopt_train(params):
    hclf = RandomForestClassifier(**params)
    hscores = cross_val_score(hclf, x_train, y_train, cv=3)
    return hscores.mean()
space = {
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 32)), 
    'min_samples_split': hp.choice('min_samples_split', range(2, 10)), 
    'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0,0.5), 
            
          }
def f(params):
    acc = hyperopt_train(params)
    return {'loss': -acc, 'status':STATUS_OK}
    
trials = Trials()
hbest = fmin(f, space, algo=tpe.suggest, max_evals=100, trials=trials)

print(hbest)


{'min_samples_leaf': 5, 'min_samples_split': 0, 'min_weight_fraction_leaf': 0.0007018032701465694}


In [15]:
#rf max_depth: 8, min_samples_split:5, min_weight_fraction:0.00302
rf_params = {
    'n_estimators': 500,
    'max_depth':8, 
    
} 
#*****
#min_samples_leaf:1, min_samples_split:5, min_weight_fraction_leaf:0.000475
et_params = {
    'n_estimators':500,
     
    'min_weight_fraction_leaf':0.000475
}

#best gb_params: max_depth = 3, min_samples_leaf = 8
gb_params = {
    'n_estimators': 500,
    'max_depth':3, 
    'min_samples_leaf':8,
    
}

#Tuned: C=0.01019, gamma:7.88695, kernel:linear
svc_params = {
    'kernel' : 'linear',
    'C' : 0.01019,
    'gamma':7.88695
    }

#base_score:0.27914, max_depth:4, min_child_weight:2, objective:multi:softmax
xgb_params = {
    'n_estimators': 10000,  
    
    'max_depth':4,
    
    'num_class':NUMCLASS,
    'objective': 'multi:softmax'
     }
#leaf_size:8, n_neighbors:1, p:2
kn_params = {
    'n_neighbors': 2,
    'leaf_size':8,
    'p':2
}

cnn_params = {
    'batch_size': 128, 
    'epochs': 50, 
    'verbose': 0
}

In [16]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)

gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)
kn = SklearnHelper(clf=KNeighborsClassifier, params=kn_params)

xg = XgbHelper(seed=SEED, params=xgb_params)


In [17]:
#ctrain_x = x_train.reshape(x_train.shape[0], 8, 8, 1)
ctest_x = x_test.reshape(x_test.shape[0], 8, 8, 1)

coof_train = np.zeros((ntrain,))
coof_test = np.zeros((ntest,))
coof_test_skf = np.empty((NFOLDS, ntest))

for i, (train_index, test_index) in enumerate(kf):
    x_tr = x_train[train_index]
    y_tr = y_train[train_index]
    x_te = x_train[test_index]
    
    cnnmodel.fit(x_tr.reshape(x_tr.shape[0],8,8,1),
                 to_categorical(y_tr), batch_size=64, epochs=50, verbose=0)
    
    opred = cnnmodel.predict(x_te.reshape(x_te.shape[0],8,8,1))
    coof_train[test_index] = [np.argmax(x) for x in opred]
    preds = cnnmodel.predict(ctest_x)
    coof_test_skf[i, :] = [np.argmax(x) for x in preds]
    
list_vals = coof_test_skf.mean(axis=0)
coof_test[:] = [round(x, 0) for x in list_vals]
coof_train = coof_train.reshape(-1, 1) 
coof_test = coof_test.reshape(-1, 1)

coof_test[:2]

array([[ 6.],
       [ 3.]])

In [18]:
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) 
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) 
#data.ravel() to remove array brackets

gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) 
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) 
kn_oof_train, kn_oof_test = get_oof(kn, x_train, y_train, x_test)

xg_oof_train, xg_oof_test = get_oof(xg, x_train, y_train, x_test)
print("Training is complete")

Training is complete


In [19]:
"""
print("et:{}".format(accuracy_score(y_test, et_oof_test)))
print("rf:{}".format(accuracy_score(y_test, rf_oof_test)))
print("gb:{}".format(accuracy_score(y_test, gb_oof_test)))
print("svc:{}".format(accuracy_score(y_test, svc_oof_test)))
print("knn: {}".format(accuracy_score(y_test, kn_oof_test)))
print("xg:{}".format(accuracy_score(y_test, xg_oof_test)))
print("cnn:{}".format(accuracy_score(y_test, coof_test)))
allDone()
"""
xg_oof_train[:2]

array([[ 6.],
       [ 5.]])

In [20]:
#f1_score(y_test, svc_oof_test, average="weighted")

In [21]:

#precision_score(y_test, svc_oof_test, average="weighted")

In [22]:
print("Sizes of {0} train, to preds {1}".format(train.shape, rf_oof_train.shape))

Sizes of (4000, 64) train, to preds (4000, 1)


In [23]:
x_trained = np.concatenate((coof_train, svc_oof_train,et_oof_train,kn_oof_train,  
            rf_oof_train, gb_oof_train),axis=1)
x_tested = np.concatenate((coof_test, svc_oof_test,et_oof_test, kn_oof_test, rf_oof_test, gb_oof_test), axis=1)
print("The change in shape from {0} to {1}".format(x_train.shape, x_trained.shape))

The change in shape from (4000, 64) to (4000, 6)


In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': range(1, 11, 1), 'n_estimators':[100,200,500,100]}
xgb_model = xgb.XGBClassifier()

xfold= KFold(len(x_trained), n_folds=3, random_state=SEED, shuffle=True)
grid_result = GridSearchCV(xgb_model, param_grid, cv=xfold)
grid_result.fit(x_trained, y_train)

#CROSS VALIDATION SCORE
print(grid_result.best_score_)
print(grid_result.best_params_)

0.994
{'max_depth': 1, 'n_estimators': 100}


In [25]:
stacked = xgb.XGBClassifier(n_estimators=1000,max_depth=2).fit(x_trained, y_train)
predictions = stacked.predict(x_tested)
#accuracy_score(y_test, predictions)      

In [26]:
#f1_score(y_test, predictions, average="weighted")

In [27]:
ids = [x for x in range(0, test_data.shape[0])]
digit_test_labelled = pd.DataFrame({'id': ids, 'labels':predictions})
digit_test_labelled.head()

,id,labels
0,0,6
1,1,3
2,2,9
3,3,2
4,4,5


In [28]:
#digit_test_labelled.to_csv('/home/steve/Desktop/revised/digits_test_labelled.csv')
